In [66]:
import pandas as pd
import numpy as np


In [67]:
# https://github.com/jamesqo/gun-violence-data
DATOS = pd.read_csv('./gun_violence_dataset/gun_violence_dataset.csv')
DATOS.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,...,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,...,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


Veamos el número de registros vacíos por cada columna:

In [68]:
# 240k filas
DATOS.isna().sum()


incident_id                         0
date                                0
state                               0
city_or_county                      0
address                         16497
n_killed                            0
n_injured                           0
incident_url                        0
source_url                        468
incident_url_fields_missing         0
congressional_district          11944
gun_stolen                      99498
gun_type                        99451
incident_characteristics          326
latitude                         7923
location_description           197588
longitude                        7923
n_guns_involved                 99451
notes                           81017
participant_age                 92298
participant_age_group           42119
participant_gender              36362
participant_name               122253
participant_relationship       223903
participant_status              27626
participant_type                24863
sources     

Eliminamos las columnas con mayor cantidad de valores nulos (participant_relationship, location_description), que son prescindibles o contienen información personal que no nos interesa. 

Eliminamos también incident_url_fields_missing que siempre tiene valor False y las columnas que tienen las url de las noticias a excepción de incident_url que no tiene ningún valor faltante (sources, source_url). 

Por último, prescindimos también de las columnas state_house_district y state_senate_district, a favor de congressional_district, que es la que menor número de valores nulos tiene y porque las tres columnas se refieren a lo mismo: la entidad territorial que elige a un solo miembro del congreso.

In [69]:
del DATOS["participant_relationship"], DATOS["location_description"], DATOS["incident_url_fields_missing"], DATOS["sources"], DATOS["source_url"], DATOS["state_house_district"], DATOS["state_senate_district"]
DATOS.isna().sum()

incident_id                      0
date                             0
state                            0
city_or_county                   0
address                      16497
n_killed                         0
n_injured                        0
incident_url                     0
congressional_district       11944
gun_stolen                   99498
gun_type                     99451
incident_characteristics       326
latitude                      7923
longitude                     7923
n_guns_involved              99451
notes                        81017
participant_age              92298
participant_age_group        42119
participant_gender           36362
participant_name            122253
participant_status           27626
participant_type             24863
dtype: int64

A continuación, vamos a crear nuevas columnas que resuman la información contenida en celdas con varios valores.

In [70]:
# Devuelve el conjunto de objectos de una celda
def get_elements_as_list(cell):
#     print(cell)
    if('||' in cell):
        return cell.split('||')
    else:
        return cell.split('|')

# Devuelve el número de elementos
def get_num_elements_of_object(cell):
    elements = get_elements_as_list(cell)
    return len(elements)     

# Devuelve un map donde key=i y value=value
def get_elements(cell):
    res = {}
    elements = get_elements_as_list(cell)
    i = 0
    for element in elements:
#         print(element)
        if '::' in element:
            value = element.split('::')[1]
        else:
            value = element.split(':')[1]
        res[i] = value
        i = i + 1
    return res

# Devuelve los elementos (get_elements) como números
def get_elements_as_numbers(cell):
    res = get_elements(cell)
    
    # Convert to number
    for i in range(0, len(res)):
        res[i] = float(res[i])
        
    return res

# Calcula el valor medio de una columna con objetos de tipo número
def compute_average_value(cell):
    if not pd.isnull(cell):
        elements = get_elements_as_numbers(cell)
        ac = 0.0
        
        for i in range(0, len(elements)):
            ac = ac + elements[i]
            
        return ac/len(elements)
    else:
        return None

# Get number of victims and suspects
def get_number_of_value(cell, value):
    elements = get_elements(cell)
    
    n_values = 0
    
    for i in range(0, len(elements)):
        if(elements[i].lower() == value.lower()):
            n_values = n_values + 1
    
    return n_values

# Returns true if an object contains part of a string
def get_contains(cell, value):
    if not pd.isnull(cell):
        elements = get_elements_as_list(cell)
        
        for i in range(0, len(elements)):
            if value in elements[i].lower():
                return True
        
        return False
    else:
        return None

def get_contains_multiple(cell, values):
    if not pd.isnull(cell):
        res = False
        for value in values:
            res = get_contains(cell, value)
            
            if res:
                return res
        
        return res
    else:
        return None

# Returns true if an object contains exactly a string
def get_contains_strict(cell, value):
    
    elements = get_elements_as_list(cell)
    
    for i in range(0, len(elements)):
        if value == elements[i].lower():
            return True
    
    return False

# Returns true if an object contains exactly one element of a list of strings
def get_contains_strict_multiple(cell, values):
    if not pd.isnull(cell):
        res = False
        for value in values:
            res = get_contains_strict(cell, value)
            
            if res:
                return res
        
        return res
    else:
        return None

# Imprime un valor si este no aparece en la lista pasada por parámetro, se usa para encontrar valores no contemplados en parámetros de tipo enumerado
def print_not_considered(cell, values, simple_list=False):
    
    if simple_list:
        elements = get_elements_as_list(cell)
    else:
        elements = get_elements(cell)
        
#     print(elements)
    
    for i in range(0, len(elements)):
        if elements[i].lower() not in values:
#             print('Values: {}'.format(values))
#             print('Element:{}'.format(elements[i]))
            print(elements[i])

# Similar to print_not_considered, but with substrings
def print_not_considered_contains(cell, values):
    
    elements = get_elements_as_list(cell)
             
            
    for i in range(0, len(elements)):
        contains = False
        for value in values:
            if value in elements[i].lower():
                contains = True
        
        if not contains:
            print(elements[i])

## 1.- Edad media de los participantes

In [71]:
DATOS['average_age'] = DATOS['participant_age'].apply(compute_average_value)

## 2.- Número de víctimas

In [72]:
# Antes de nada, comprobamos que no existen más valores en la columna participant_type, exceptuando Victim y subject-suspect
def print_not_considered_participant_type(cell):
    if not pd.isnull(cell):
        return print_not_considered(cell, ['victim', 'subject-suspect'])
    else:
        return None

DATOS['participant_type'].apply(print_not_considered_participant_type)

def get_num_victims(cell):
    if not pd.isnull(cell):
        return get_number_of_value(cell, 'Victim')
    else:
        return None

DATOS['n_victims'] = DATOS['participant_type'].apply(get_num_victims)

## 3.- Número de sospechosos 

In [73]:
def get_num_suspects(cell):
    if not pd.isnull(cell):
        return get_number_of_value(cell, 'Subject-Suspect')
    else:
        return None

DATOS['n_suspects'] = DATOS['participant_type'].apply(get_num_suspects)

## 4.- Convertir gun_stolen a boolean
True si alguna de las armas ha sido robada, False en el caso contrario o Unknown si no se sabe. Borrar esta columna, ya forma parte de incident-type

In [79]:
def print_not_considered_gun_stolen(cell):
    if not pd.isnull(cell):
        if "not-stolen" in cell.lower():
            return False
        else:
            if "stolen" in cell.lower():
                return True
            else:
                return "Unknown"
    else:
        return "Unknown"

DATOS["stolen_gun_involved"] = DATOS['gun_stolen'].apply(print_not_considered_gun_stolen)

## 5.- Crear boolean minors_involved

In [75]:
def print_not_considered_participant_age_group(cell):
    if not pd.isnull(cell):
        return print_not_considered(cell, ['adult 18+', 'teen 12-17', 'child 0-11'])
    else:
        return None

DATOS['participant_age_group'].apply(print_not_considered_participant_age_group)

DATOS["minors_involved"] = DATOS['participant_age_group'].apply(get_contains_multiple, values=['teen 12-17', 'child 0-11'])

## 6.- Porcentaje de mujeres involucradas

In [94]:
def get_female_percentage(cell):
    if not pd.isnull(cell):
        genders = list(get_elements(cell).values())
        female = genders.count("Female")
        male = genders.count("Male")
        return female/(len(genders))*100

    else:
        return None

DATOS["women_percentage"] = DATOS['participant_gender'].apply(get_female_percentage)

## 7.- Convertir tipos de incidentes en booleans

In [76]:
def get_incident_types():
    return [
                                    'Domestic Violence'.lower(),
                                        'Drive-by'.lower(),
                                        'Gang involvement'.lower(),
                                        'Home Invasion'.lower(),
                                        'Institution/Group/Business'.lower(),
                                        'Shooting'.lower(),
                                        'Murder'.lower(),
                                        'Officer Involved'.lower(),
                                        'Possession'.lower(),
                                        'Shot'.lower(),
                                        'Suicide'.lower(),
                                    'Drug involvement'.lower(),
                                    'Stolen/Illegally owned gun'.lower(),
                                    'Bar/club incident'.lower(),
                                    'Sex crime'.lower(),
                                    'School Incident'.lower(),
                                    'Kidnapping/abductions/hostage'.lower(),
                                    'Defensive Use'.lower(),
                                    'Car-jacking'.lower(),
                                    'Armed robbery'.lower(),
                                    'Hate crime'.lower(),
                                    'Child'.lower(),
                                    'Accidental/Negligent Discharge'.lower(),
                                    'Brandishing/flourishing/open carry/lost/found'.lower(),
                                    'Gun(s) stolen'.lower(),
                                    'Guns stolen'.lower(), # Group with the previous one
                                    'Road rage'.lower(),
                                    'Under the influence of alcohol or drugs'.lower(),
                                    'BB/Pellet/Replica gun'.lower(),
                                    'Cleaning gun'.lower(),
                                    'Implied Weapon'.lower(),
                                    'House party'.lower(),
                                    'ATF/LE Confiscation/Raid/Arrest'.lower(),
                                    'Gun at school'.lower(),
                                    'Thought gun was unloaded'.lower(),
                                    'Hunting accident'.lower(),
                                    'stolen gun'.lower(),
                                    'Police Targeted'.lower(),
                                    'Pistol-whipping'.lower(),
                                    'Playing with gun'.lower(),
                                    'Shootout'.lower(),
                                    'Unlawful purchase/sale'.lower(),
                                    'Non-Aggression'.lower(),
                                    'Gun shop robbery or burglary'.lower(),
                                    'Concealed Carry License'.lower(),
                                    'Assault weapon'.lower(),
                                    'LOCKDOWN/ALERT'.lower(),
                                    'TSA Action'.lower(),
                                    'Terrorism'.lower(),
                                    'Ghost gun'.lower(),
                                    'Political Violence'.lower(),
                                    'Mistaken ID'.lower(),
                                    'NAV'.lower(),
                                    'Gun buy back action'.lower()
                                ]

# Antes de nada, comprobamos que no existen más valores en la columna participant_type, exceptuando Victim y subject-suspect
def print_not_considered_incident_characteristics(cell):
    if not pd.isnull(cell):
        incident_types = get_incident_types()
        return print_not_considered_contains(cell, 
                                    incident_types
                                )
    else:
        return None

DATOS['incident_characteristics'].apply(print_not_considered_incident_characteristics)

for incident_type in get_incident_types():
    DATOS[incident_type] = DATOS['incident_characteristics'].apply(get_contains, value=incident_type)

In [95]:
DATOS.to_csv("./gun_violence_dataset/processed_gun_violence_dataset.csv")
# DATOS.to_excel("./gun_violence_dataset/processed_gun_violence_dataset.xlsx")

In [78]:
DATOS.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,congressional_district,gun_stolen,...,concealed carry license,assault weapon,lockdown/alert,tsa action,terrorism,ghost gun,political violence,mistaken id,nav,gun buy back action
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,14.0,NaN,...,False,False,False,False,False,False,False,False,False,False
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,43.0,NaN,...,False,False,False,False,False,False,False,False,False,False
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,9.0,0::Unknown||1::Unknown,...,False,False,False,False,False,False,False,False,False,False
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,6.0,NaN,...,False,False,False,False,False,False,False,False,False,False
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,6.0,0::Unknown||1::Unknown,...,False,False,False,False,False,False,False,False,False,False


Una vez eliminadas las columnas con mayor número de valores faltantes y aquellas que no nos son de interés, y creadas columnas con nueva información, procedemos a dividir el dataset en tres subdatasets, cada uno de los cuales será tratado en cuanto a valores nulos y faltantes; en lugar de hacer esto antes de dividirlo pues se perdería información.

# Dataset de personas

Extraemos la información de las personas involucradas en un dataset aparte. Registraremos:
* Name
* Age
* Age_group
* Gender
* Status
* Type
* State
* City_or_county
* N_killed
* N_injured
* Average_age
* N_victims
* N_suspects
* Stolen_gun_involved
* Minors_involved

In [89]:

people = pd.DataFrame(columns=['Name', 'Age', 'Age_group', 'Gender', 'Status', 'Type', 'Incident_id', 'State', 'City_or_county', 'N_killed', 'N_injured', "Average_age", "N_victims", "N_suspects", "Stolen_gun_involved", "Minors_involved"])

counter = 0

for index, row in DATOS.iterrows():
    average_age = row["average_age"] if not pd.isnull(row["average_age"]) else None
    n_victims = row["n_victims"] if not pd.isnull(row["n_victims"]) else None
    n_suspects = row["n_suspects"] if not pd.isnull(row["n_suspects"]) else None
    stolen_gun_involved = row["stolen_gun_involved"] if not pd.isnull(row["stolen_gun_involved"]) else None
    minors_involved = row["minors_involved"] if not pd.isnull(row["minors_involved"]) else None
    types = get_elements(row["participant_type"]) if not pd.isnull(row["participant_type"]) else {}
    statuses = get_elements(row["participant_status"]) if not pd.isnull(row["participant_status"]) else {}
    genders = get_elements(row["participant_gender"]) if not pd.isnull(row["participant_gender"]) else {}
    age_groups = get_elements(row["participant_age_group"]) if not pd.isnull(row["participant_age_group"]) else {}
    ages = get_elements(row["participant_age"]) if not pd.isnull(row["participant_age"]) else {}
    participant_names = get_elements(row["participant_name"]) if not pd.isnull(row["participant_name"]) else {}

    participants_ids = list(types.keys()) + list(statuses.keys()) + list(genders.keys()) + list(age_groups.keys()) + list(ages.keys()) + list(participant_names.keys())
    participants_ids = list(dict.fromkeys(participants_ids)) # delete duplicates

    for i in participants_ids:
        name = participant_names[i] if i in participant_names.keys() else None
        age = ages[i] if i in ages.keys() else None
        age_group = age_groups[i] if i in age_groups.keys() else None
        gender = genders[i] if i in genders.keys() else None
        status = statuses[i] if i in statuses.keys() else None
        tyype = types[i] if i in types.keys() else None

        people = people.append(pd.Series(data={"Name":name,"Age":age,"Age_group":age_group,"Gender":gender,"Status":status,"Type":tyype,"Incident_id":row["incident_id"],"State":row["state"],
            "City_or_county":row["city_or_county"], "N_killed":row["n_killed"], "N_injured":row["n_injured"], "Average_age": average_age, "N_victims": n_victims, 
            "N_suspects":n_suspects, "Stolen_gun_involved": stolen_gun_involved, "Minors_involved": minors_involved}), ignore_index=True)
    counter = counter + 1

    if counter == 10:
        break

people.head(20)


,Name,Age,Age_group,Gender,Status,Type,Incident_id,State,City_or_county,N_killed,N_injured,Average_age,N_victims,N_suspects,Stolen_gun_involved,Minors_involved
0,Julian Sims,20,Adult 18+,Male,Arrested,Victim,461105,Pennsylvania,Mckeesport,0,4,20.0,4.0,1.0,Unknown,False
1,None,None,Adult 18+,Male,Injured,Victim,461105,Pennsylvania,Mckeesport,0,4,20.0,4.0,1.0,Unknown,False
2,None,None,Adult 18+,Male,Injured,Victim,461105,Pennsylvania,Mckeesport,0,4,20.0,4.0,1.0,Unknown,False
3,None,None,Adult 18+,Female,Injured,Victim,461105,Pennsylvania,Mckeesport,0,4,20.0,4.0,1.0,Unknown,False
4,None,None,Adult 18+,None,Injured,Subject-Suspect,461105,Pennsylvania,Mckeesport,0,4,20.0,4.0,1.0,Unknown,False
5,Bernard Gillis,20,Adult 18+,Male,Killed,Victim,460726,California,Hawthorne,1,3,20.0,4.0,1.0,Unknown,False
6,None,None,Adult 18+,None,Injured,Victim,460726,California,Hawthorne,1,3,20.0,4.0,1.0,Unknown,False
7,None,None,Adult 18+,None,Injured,Victim,460726,California,Hawthorne,1,3,20.0,4.0,1.0,Unknown,False
8,None,None,Adult 18+,None,Injured,Victim,460726,California,Hawthorne,1,3,20.0,4.0,1.0,Unknown,False
9,None,None,None,None,None,Subject-Suspect,460726,California,Hawthorne,1,3,20.0,4.0,1.0,Unknown,False


In [ ]:
# TODO: Status y type dividirlos en One-Hot Encoding
# TODO: Tratar valores nulos

In [88]:
people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Name                 28 non-null     object 
 1   Age                  26 non-null     object 
 2   Age_group            45 non-null     object 
 3   Gender               41 non-null     object 
 4   Status               51 non-null     object 
 5   Type                 53 non-null     object 
 6   State                53 non-null     object 
 7   City_or_county       53 non-null     object 
 8   N_killed             53 non-null     object 
 9   N_injured            53 non-null     object 
 10  Average_age          42 non-null     object 
 11  N_victims            53 non-null     float64
 12  N_suspects           53 non-null     float64
 13  Stolen_gun_involved  53 non-null     object 
 14  Minors_involved      47 non-null     object 
dtypes: float64(2), object(13)
memory usage: 6.

# Dataset de incidentes

In [ ]:
# TODO: Eliminar columnas de personas y eso que ya se han tratado y tratar los valores nulos